In [ ]:
!pip install -U langchain-ollama

## Create a list of unique scenarios

In [ ]:
import pandas as pd

scenarios_file = "scenarios.csv"
unique_scenarios_file = "scenarios_unique.csv"

def select_unique_scenarios(input, output):
    df = pd.read_csv(input, delimiter=";")

    df['Scenario ID'] = df['Scenario ID'].str.strip()
    df['User'] = df['User'].str.strip()

    unique = df.drop_duplicates(subset=['Scenario ID'], keep='first')[
        ['Scenario ID', 'User']]
   
    unique.to_csv(output, index=False, sep=";")

    print(f"Unique scenarios saved to {output}")


select_unique_scenarios(scenarios_file, unique_scenarios_file)

## Generate prompts

In [ ]:
import pandas as pd

def generate_llm_prompt(scenario_id, scenarios_df, threats_df, vulnerabilities_df):
    """
    Generates a formatted LLM prompt based on the given scenario, threats, and vulnerabilities.
    
    Args:
        scenario_id (str): The scenario ID to pull the correct description.
        scenarios_df (pd.DataFrame): DataFrame containing scenarios data.
        threats_df (pd.DataFrame): DataFrame containing threat data.
        vulnerabilities_df (pd.DataFrame): DataFrame containing vulnerability data.
        
    Returns:
        str: The LLM prompt formatted as a string.
    """
    scenario_row = scenarios_df[scenarios_df['Scenario ID'] == scenario_id]
    if not scenario_row.empty:
        scenario_description = scenario_row['User'].iloc[0] 
    else:
        print(f"No scenario found with ID: {scenario_id}")
    
    risk_description = scenario_row['Assistant - Risk description'] if 'Assistant - Risk description' in scenario_row else "No risk description available."
    vulnerability_description = scenario_row['Assistant - Vulnerability description'] if 'Assistant - Vulnerability description' in scenario_row else "No vulnerability description available."
    
    threats_text = "\n\n".join([
    f"ThreatID: {row['THREAT ID']}\n"
    f"ThreatName: {row['THREAT']}\n"
    f"ThreatDescription: {row['DESCRIPTION']}\n"
    "---------------------------------"  
    for _, row in threats_df.iterrows()
])

    vulnerabilities_text = "\n\n".join([
    f"VulnID: {row['ID']}\n"
    f"VulnerabilityName: {row['VULNERABILITY']}\n"
    f"VulnDescription: {row['DESCRIPTION']}\n"
    "---------------------------------"  
    for _, row in vulnerabilities_df.iterrows()
])

    prompt = f"""
    
    ROLE: You are an assistant specializing in security risk analysis. Your task is to assess the provided scenario and identify any potential threats from the given list.
    Do not generate new threats. Use values that I provide

### **Instructions:**

1. **Read the scenario carefully.** This is the only scenario you need to evaluate. Do not invent any details or make assumptions. Always respond in English.

    - ScenarioID: {scenario_id}
    - Scenario description: "{scenario_description}"

2. **Identify all the applicable threats.** Review the scenario and compare it against the "Threats List" below. If a security threat is present, please explain what the security threat is. Only choose threats that are exact matches from the list. Do not select any threats that are not in the provided list.
    - **Threats List:**
    {threats_text}

    **DO NOT create new threats or mention anything outside of the provided Threats List.** If no threats apply, leave the "Threats" array empty.

3. **Create a new JSON object for every threat you find.** For each identified threat, create a JSON object with the following format:

    {{
        "ScenarioID": "{scenario_id}",
        "Scenario": "{scenario_description}",
        "Threats": [
            {{
                "ThreatID": "<THREAT_ID>",
                "ThreatName": "<THREAT_NAME>",
                "ThreatDescription": "<THREAT_DESCRIPTION>"
            }}
        ]
    }}

    **For Example**
    {{
        "ScenarioID": "{scenario_id}",
        "Scenario": "{scenario_description}",
        "Threats": [
            {{
                "ThreatID": "T2"
                "ThreatName": "Power supply"
                "ThreatDescription": "Power failure to devices which may cause loss or corruption of processed data"
            }}
        ]
    }}


    - Ensure that Threats matches list i provided.

    ### **Important Notes:**
    - **Do not** generate new threats. Use only the values provided in the "Threats List".
    - Ensure that the JSON output is valid and correctly formatted.
    """
    
    return prompt


def generate_prompts_for_all_scenarios(scenarios_df, threats_df, vulnerabilities_df):
    """
    Generate LLM prompts for all scenarios in the CSV.

    Args:
        scenarios_df (pd.DataFrame): DataFrame containing scenarios.
        threats_df (pd.DataFrame): DataFrame containing threats.
        vulnerabilities_df (pd.DataFrame): DataFrame containing vulnerabilities.

    Returns:
        list: A list of prompts for each scenario.
    """
    prompts = []
    for scenario_id in scenarios_df['Scenario ID']:
        prompt = generate_llm_prompt(scenario_id, scenarios_df, threats_df, vulnerabilities_df)
        prompts.append(prompt)
    
    return prompts





scenarios_df = pd.read_csv(unique_scenarios_file, delimiter=';')
threats_df = pd.read_csv('threat.csv', delimiter=';')
vulnerabilities_df = pd.read_csv('vulnerability.csv', delimiter=';')

scenarios_df.columns = scenarios_df.columns.str.strip()
threats_df.columns = threats_df.columns.str.strip()
vulnerabilities_df.columns = vulnerabilities_df.columns.str.strip()

prompts = generate_prompts_for_all_scenarios(scenarios_df, threats_df, vulnerabilities_df)

print(prompts[1])  # Example: Print the first prompt


## Generate answers

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_ollama.llms import OllamaLLM
import csv

answer_limit = 193
csv_filename = "answers.csv"

template = """Question: {question}
Answer: Return only json"""
prompt = ChatPromptTemplate.from_template(template)

def generate_answer_for_prompt(question):
    model = OllamaLLM(model="qwen2.5")  
    chain = prompt | model
    answer = chain.invoke({"question": question})
    return answer

def generate_answers_for_all_prompts(prompts, limit, answer_file):
    limit = min(limit, len(prompts))

    for i in range(limit):
        question = prompts[i]
        answer = generate_answer_for_prompt(question)  

        with open(answer_file, mode="a", newline="", encoding="utf-8") as file:
            writer = csv.writer(file)
            writer.writerow([answer]) 

        print(f"Answer saved to {answer_file}")


generate_answers_for_all_prompts(prompts, answer_limit, csv_filename)



In [ ]:
import csv
import json
import re

input_file = "answers.csv"
output_file = "formatted_answer.csv"

with open(input_file, "r", encoding="utf-8") as file:
    content = file.read().strip()

json_entries = re.findall(r'```json\n(.*?)\n```', content, re.DOTALL)

data = []

for json_str in json_entries:
    try:
        json_str = json_str.replace('""', '"')

        parsed_data = json.loads(json_str)

        scenario_id = parsed_data.get("ScenarioID", "")
        scenario = parsed_data.get("Scenario", "")

        if parsed_data.get("Threats"):
            for threat in parsed_data["Threats"]:
                data.append([
                    scenario_id,
                    scenario,
                    threat.get("ThreatID", ""),
                    threat.get("ThreatName", ""),
                    threat.get("ThreatDescription", "")
                ])
        else:
            data.append([scenario_id, scenario, "", "", ""])  

    except json.JSONDecodeError as e:
        print(f"Skipping invalid JSON: {json_str}")
        print(f"Error: {e}")

with open(output_file, "w", newline="", encoding="utf-8") as file:
    writer = csv.writer(file, delimiter=';')
    writer.writerow(["ScenarioID", "Scenario", "ThreatID", "ThreatName", "ThreatDescription"])
    writer.writerows(data)

print(f"Formatted CSV file saved as {output_file}")


In [ ]:
import pandas as pd

def generate_llm_prompt1(scenario_id, scenarios_df, vulnerabilities_df):
    """
    Generates a formatted LLM prompt based on the given scenario, pre-identified threats, and possible vulnerabilities.

    Args:
        scenario_id (str): The scenario ID to pull the correct description.
        scenarios_df (pd.DataFrame): DataFrame containing scenarios and threats.
        vulnerabilities_df (pd.DataFrame): DataFrame containing vulnerability data.

    Returns:
        str: The LLM prompt formatted as a string.
    """
    
    scenario_row = scenarios_df[scenarios_df['ScenarioID'] == scenario_id]
    if not scenario_row.empty:
        scenario_description = scenario_row['Scenario'].iloc[0]
        threat_id = scenario_row['ThreatID'].iloc[0]
        threat_name = scenario_row['ThreatName'].iloc[0]
        threat_description = scenario_row['ThreatDescription'].iloc[0]
    else:
        print(f"No scenario found with ID: {scenario_id}")
        return None 

    
    vulnerabilities_text = "\n\n".join([
        f"VulnID: {row['ID']}\n"
        f"VulnerabilityName: {row['VULNERABILITY']}\n"
        f"VulnDescription: {row['DESCRIPTION']}\n"
        "---------------------------------"
        for _, row in vulnerabilities_df.iterrows()
    ])

    
    prompt = f"""
    ROLE: You are an assistant specializing in security risk analysis. Your task is to assess the provided scenario and identify any applicable **vulnerabilities** from the given list.
    The threats have already been identified.

### **Instructions:**

1. **Read the scenario carefully.** This is the only scenario you need to evaluate. Do not invent any details or make assumptions. Always respond in English.

    - ScenarioID: {scenario_id}
    - Scenario description: "{scenario_description}"

2. **This is the already identified threat for this scenario:**
    ThreatID: {threat_id}
    ThreatName: {threat_name}
    ThreatDescription: {threat_description}

3. **Identify all the applicable vulnerabilities.** Review the scenario and compare it against the "Vulnerabilities List" below. If a vulnerability is present, explain why it applies. Only select vulnerabilities that match exactly from the provided list.
    - **Vulnerabilities List:**
    {vulnerabilities_text}

    **DO NOT create new vulnerabilities or mention anything outside of the provided list.** If no vulnerabilities apply, leave the "Vulnerabilities" array empty.

4. **Create a new JSON object for every identified vulnerability**, using the following format:

    {{
        "ScenarioID": "{scenario_id}",
        "Scenario": "{scenario_description}",    
        "ThreatID": "{threat_id}",
        "ThreatName": "{threat_name}",
        "ThreatDescription": "{threat_description}"
        "Vulnerabilities": [
            {{
                "VulnID": "<VULN_ID>",
                "VulnerabilityName": "<VULNERABILITY_NAME>",
                "VulnDescription": "<VULN_DESCRIPTION>"
            }}
        ]
    }}

    **Example Output**
    {{
        "ScenarioID": "{scenario_id}",
        "Scenario": "{scenario_description}",    
        "ThreatID": "{threat_id}",
        "ThreatName": "{threat_name}",
        "ThreatDescription": "{threat_description}"
        "Vulnerabilities": [
            {{
                "VulnID": "V1",
                "VulnerabilityName": "Communication channels not adequately protected",
                "VulnDescription": "Channels (carrying classified/sensitive corporate data) not subject to security procedures (encryption devices) or not physically inaccessible (e.g. connections with accessible data cables, to which interception tools can be connected)"
            }}
        ]
    }}

    - Ensure that **vulnerabilities match exactly** from the provided list.

### **Important Notes:**
- **Do not** generate new vulnerabilities. Use only the values provided in the "Vulnerabilities List".
- Ensure that the JSON output is valid and correctly formatted.
"""

    return prompt


def generate_prompts_for_all_scenarios1(scenarios_df, vulnerabilities_df):
    """
    Generate LLM prompts for all scenarios.

    Args:
        scenarios_df (pd.DataFrame): DataFrame containing scenarios and threats.
        vulnerabilities_df (pd.DataFrame): DataFrame containing vulnerabilities.

    Returns:
        list: A list of prompts for each scenario.
    """
    prompts = []
    for scenario_id in scenarios_df['ScenarioID']:
        prompt = generate_llm_prompt1(scenario_id, scenarios_df, vulnerabilities_df)
        if prompt:  
            prompts.append(prompt)
    
    return prompts



scenarios_df = pd.read_csv("formatted_answer.csv", delimiter=';')
vulnerabilities_df = pd.read_csv("vulnerability.csv", delimiter=';')


scenarios_df.columns = scenarios_df.columns.str.strip()
vulnerabilities_df.columns = vulnerabilities_df.columns.str.strip()


prompts_vulnerabilties = generate_prompts_for_all_scenarios1(scenarios_df, vulnerabilities_df)


print(prompts_vulnerabilties[0])  


In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_ollama.llms import OllamaLLM
import csv


answer_limit = len(prompts_vulnerabilties)  
csv_filename = "answers1.csv"  

template = """Question: {question}
Answer: Return only JSON"""
prompt = ChatPromptTemplate.from_template(template)

def generate_answer_for_prompt1(question):
    """
    Invokes the LLM with a given question prompt and returns the JSON answer.
    """
    model = OllamaLLM(model="qwen2.5")  
    chain = prompt | model
    answer = chain.invoke({"question": question}) 
    return answer

def generate_answers_for_all_prompts1(prompts, limit, answer_file):
    """
    Iterates through a list of prompts, gets responses from the LLM, and stores them in a CSV file.
    """
    limit = min(limit, len(prompts))

    for i in range(limit):
        question = prompts[i]  
        answer = generate_answer_for_prompt1(question)  

        with open(answer_file, mode="a", newline="", encoding="utf-8") as file:
            writer = csv.writer(file)
            writer.writerow([answer]) 

        print(f"Answer saved to {answer_file}")

generate_answers_for_all_prompts1(prompts_vulnerabilties, answer_limit, csv_filename)


In [ ]:
import csv
import json
import re

input_file = "answers1.csv" 
output_file = "final.csv"

with open(input_file, "r", encoding="utf-8") as file:
    content = file.read().strip()

json_entries = re.findall(r'```json\n(.*?)\n```', content, re.DOTALL)

data = []

for json_str in json_entries:
    try:
        json_str = json_str.replace('""', '"')

        parsed_data = json.loads(json_str)

        scenario_id = parsed_data.get("ScenarioID", "")
        scenario_desc = parsed_data.get("Scenario", "")
        threat_id = parsed_data.get("ThreatID", "")
        threat_name = parsed_data.get("ThreatName", "")
        threat_desc = parsed_data.get("ThreatDescription", "")

        if "Vulnerabilities" in parsed_data and parsed_data["Vulnerabilities"]:
            for vuln in parsed_data["Vulnerabilities"]:
                data.append([
                    scenario_id,
                    scenario_desc,
                    threat_id,
                    threat_name,
                    threat_desc,
                    vuln.get("VulnID", ""),
                    vuln.get("VulnerabilityName", ""),
                    vuln.get("VulnDescription", "")
                ])
        else:
            data.append([scenario_id, scenario_desc, threat_id, threat_name, threat_desc, "", "", ""])

    except json.JSONDecodeError as e:
        print(f"Skipping invalid JSON: {json_str}")
        print(f"Error: {e}")

with open(output_file, "w", newline="", encoding="utf-8") as file:
    writer = csv.writer(file, delimiter=';')
    writer.writerow([
        "ScenarioID", "Scenario", "ThreatID", "ThreatName", "ThreatDescription",
        "VulnID", "VulnerabilityName", "VulnDescription"
    ])
    writer.writerows(data)

print(f"Formatted CSV file saved as {output_file}")
